<a href="https://colab.research.google.com/github/Dimka27/Projects/blob/main/CUP_IT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import 

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
     |████████████████████████████████| 870kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=554b97846f137955700f7ec469f6ed5660131555399e919489c78e780f54a4ae
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import os
import zipfile
from google.colab import files

import numpy as np
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
import torch
import torch.nn as nn
import gensim.downloader as api
import torch.optim as optim
import transformers as ppb
import warnings
import json
warnings.filterwarnings('ignore')

from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Sampler, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Download files

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip

--2021-04-06 16:57:32--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  13.0MB/s    in 4.6s    

2021-04-06 16:57:37 (19.5 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]



In [ ]:
PATH_zip_file = "/content/snli_1.0.zip"
unzip_file = zipfile.ZipFile(PATH_zip_file, 'r')
for path in [path for path in unzip_file.namelist()
            if ".txt" in path and len(path.split('/')) == 2]:
    unzip_file.extract(path)

In [ ]:
PATH_dev = "/content/snli_1.0/snli_1.0_dev.txt"
PATH_train = "/content/snli_1.0/snli_1.0_train.txt"
PATH_test = "/content/snli_1.0/snli_1.0_test.txt"
PATH_model = "/content/best_model"

In [ ]:
text = open(PATH_test)
d = text.read()
len(d.split('\n'))

10002

In [ ]:
len(d.split('\n'))

10002

# Arguments

In [ ]:
#@title Set parameters { vertical-output: true, display-mode: "form" }

word2vec = api.load('glove-wiki-gigaword-300')

lr = 2e-5 #@param
emb_dim = 300 #@param
hid_dim = 256 #@param
n_layers = 2 #@param
dropout = 0.3 #@param
batch_size = 300 #@param
num_epochs = 25 #@param
output_size = 3 #@param
hidden_size = 256 #@param

clip = 5 #@param
max_len = 256 #@param
vocab_size = len(word2vec.vocab)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

[==================================================] 100.0% 376.1/376.1MB downloaded


# Class dataset

In [ ]:
class CustDataset(Dataset):
    def __init__(self, path, word2vec, max_length, test=False):
        self.path = path
        self.test = test
        self.labels = ['gold_label', 'sentence1', 'sentence2']
        self.predict_labels = ['contradiction', 'neutral', 'entailment']
        self.label_encoder = LabelEncoder()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.stopwords = stopwords.words('english')
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        #self.word2vec = api.load('glove-twitter-50')
        self.word2vec = word2vec
        self.max_length = max_length # Лучше ограничить длину текстов
        self.mean = np.mean(self.word2vec.vectors, axis=0)
        self.std = np.std(self.word2vec.vectors, axis=0)
        self.data = self.create_dataset()
        self.stange_input = set()

        self.label_encoder.fit(self.predict_labels)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]
        
        try:
            return {
                'label': self.label_encoder.transform([text[0]])[0],
                'sentence1': self.transform_sentence(text[1]),
                'sentence2': self.transform_sentence(text[2]),
            }
        except ValueError:
            return self.__getitem__(np.random.randint(len(self.data)))
       

    def transform_sentence(self, sentence):
        tokens = [self.lemmatizer.lemmatize(t) for t in self.tokenizer.tokenize(sentence.lower())] # Получаем токены
        tokens = [t for t in tokens if not t in self.stopwords]
        embeds = [self.word2vec.get_vector(t) for t in tokens if t in self.word2vec] # Собираем эмбеддинги

        return (np.array(embeds) - self.mean) / self.std
        

    def create_dataset(self):
        text = open(self.path)

        labels = text.readline()[:-1].split('\t')  # считываем лейблы
        idx_labels = [labels.index(label) for label in self.labels] # индексы данных, которые будем использовать
        data = [] 

        for line in text:
            line = line[:-1].split('\t')
            if line[0] in self.predict_labels:
                data.append( [line[idx] for idx in idx_labels] )
            elif self.test:
                line[0] = self.predict_labels[np.random.randint(3)]
                data.append( [line[idx] for idx in idx_labels] )

        return data

In [ ]:
train_dataset = CustDataset(path=PATH_train, word2vec=word2vec, max_length=16)
dev_dataset = CustDataset(path=PATH_dev, word2vec=word2vec, max_length=16)
test_dataset = CustDataset(path=PATH_test, word2vec=word2vec, max_length=16, test=True)

# Functions for dataloader

In [ ]:
class TextSampler(Sampler):
    def __init__(self, sampler, batch_size=batch_size):
        self.sampler = sampler
        self.batch_size = batch_size

    def __iter__(self):
        batch = []
        max_len = 0
        for ix in self.sampler:
            row = self.sampler.data_source[ix]
            max_len = max(max_len, len(row['sentence1']), len(row['sentence2'])) # Обновим длину текстов
            if len(batch) == self.batch_size: #len(batch) + 1) * max_len > self.batch_size_tokens or 
                yield batch
                batch = []
                max_len = max(len(row['sentence1']), len(row['sentence2'])) # Если batch окажется слишком большим, то вернем его и обновим batch

            batch.append(ix)
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return np.ceil(len(self.sampler) / self.batch_size).astype(np.int)

In [ ]:
def get_padded(values):
    max_len = 0
    for value in values:
        if len(value) > max_len:
            max_len = len(value)

    padded = np.array([value + [0]*(max_len-len(value)) for value in values])

    return padded

In [ ]:
def collate_fn(batch):
    max_len = max(max(len(row["sentence1"]), len(row["sentence2"])) for row in batch)
    input_embeds1 = np.zeros((len(batch), max_len, word2vec.vector_size))
    input_embeds2 = np.zeros((len(batch), max_len, word2vec.vector_size))
    atten_mask1 = np.where(input_embeds1 != 0, 1, 0)
    atten_mask2 = np.where(input_embeds2 != 0, 1, 0)
    labels = np.zeros((len(batch),))
    labels1 = np.zeros((len(batch),))
    labels2 = np.zeros((len(batch),))
    labels3 = np.zeros((len(batch),))
    labels4 = np.zeros((len(batch),))
    labels5 = np.zeros((len(batch),))
    for idx, row in enumerate(batch):
        input_embeds1[idx][:len(row["sentence1"])] += row["sentence1"]
        input_embeds2[idx][:len(row["sentence2"])] += row["sentence2"]
        labels[idx] = row["label"]
    return {
        "sentence1": torch.FloatTensor(input_embeds1),
        "sentence2": torch.FloatTensor(input_embeds2),
        "labels": torch.LongTensor(labels),
        }

In [ ]:
train_sampler = TextSampler(RandomSampler(train_dataset))
train_loader = DataLoader(train_dataset, batch_sampler=train_sampler, collate_fn=collate_fn, num_workers=2)

In [ ]:
dev_sampler = TextSampler(RandomSampler(dev_dataset))
dev_loader = DataLoader(dev_dataset, batch_sampler=dev_sampler, collate_fn=collate_fn, num_workers=2)

In [ ]:
test_sampler = TextSampler(SequentialSampler(test_dataset), batch_size=len(test_dataset))
test_loader = DataLoader(test_dataset, batch_sampler=test_sampler, collate_fn=collate_fn, num_workers=2)

# Model

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, emb_dim):
        super(BiLSTM, self).__init__()

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.embedding = nn.Embedding(vocab_size, emb_dim)


        self.dropout = 0.2
        self.bilstm = nn.LSTM(self.hidden_size, self.hidden_size, dropout=self.dropout, bidirectional=True)

        self.projection = nn.Linear(self.emb_dim, self.hidden_size)


        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = self.dropout)
        self.pool = nn.AvgPool1d(3, stride=2, padding=0)

        self.lin1 = nn.Linear(self.hidden_size * 2 * 4, self.hidden_size)
        self.lin2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.lin3 = nn.Linear(self.hidden_size, self.output_size)

        for lin in [self.lin1, self.lin2, self.lin3]:
            nn.init.xavier_uniform_(lin.weight)
            nn.init.zeros_(lin.bias)

        self.out = nn.Sequential(
            self.lin1,
            self.relu,
            self.dropout,
            self.lin2,
            self.relu,
            self.dropout,
            self.lin3
        )



    def forward(self, input_sentences1, input_sentences2, batch_size=None):

        
        input1 = input_sentences1.permute(1, 0, 2)
        input2 = input_sentences2.permute(1, 0, 2)

        input1 = self.relu(self.projection(input1))
        input2 = self.relu(self.projection(input2))

        if batch_size is None:
            h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())



        _, (output1, _) = self.bilstm(input1, (h_0, c_0))
        _, (output2, _) = self.bilstm(input2, (h_0, c_0))

        
        if batch_size is None:
            output1 = output1[-2:].transpose(0, 1).contiguous().view(self.batch_size, -1)
            output2 = output2[-2:].transpose(0, 1).contiguous().view(self.batch_size, -1)
        else:
            output1 = output1[-2:].transpose(0, 1).contiguous().view(batch_size, -1)
            output2 = output2[-2:].transpose(0, 1).contiguous().view(batch_size, -1)


        output = self.out(
            torch.cat((output1, output1 * output2, torch.abs(output1 - output2), output2), dim=1)
        )
        
        return output 

# train/eval

In [ ]:
def train(model, loader, epoch, criterion, otpimizer):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    steps = 0
    len_d = 0
    model.train()
    pbar = tqdm(enumerate(loader), total=len(loader), leave=False)
    pbar.set_description(f"Epoch {epoch + 1}")
    for ind, batch in pbar:
        with torch.set_grad_enabled(True):
            emb1 = batch['sentence1']
            emb2 = batch['sentence2']
            target = batch['labels']
            target = torch.autograd.Variable(target).long()

            if torch.cuda.is_available():
                emb1 = emb1.cuda()
                emb2 = emb2.cuda()
                target = target.cuda()

            otpimizer.zero_grad()
            prediction = model(emb1, emb2, emb1.shape[0])

            loss = criterion(prediction, target)
            loss.backward()
            otpimizer.step()
                
            len_d += len(emb1)
            steps += 1
            total_epoch_loss += loss.item()
            total_epoch_acc += (torch.max(prediction, 1)[1].view(len(target)) == target).sum().item()
            

    return total_epoch_loss/len_d, 100 * total_epoch_acc/len_d

In [ ]:
def eval(model, loader, epoch, criterion):
    total_epoch_loss = 0
    total_epoch_acc = 0
    len_d = 0
    model.cuda()
    model.eval()
    pbar = tqdm(loader)
    pbar.set_description(f"Epoch {epoch + 1}")
    with torch.no_grad():
        for batch in pbar:
            emb1 = batch['sentence1']
            emb2 = batch['sentence2']
            target = batch['labels']
            target = torch.autograd.Variable(target).long()

            if torch.cuda.is_available():
                emb1 = emb1.cuda()
                emb2 = emb2.cuda()
                target = target.cuda()
    

            prediction = model(emb1, emb2, emb1.shape[0])
            len_d += len(emb1)
            loss = criterion(prediction, target)
            total_epoch_loss += loss.item()
            total_epoch_acc += (torch.max(prediction, 1)[1].view(len(target)) == target).sum().item()

    return total_epoch_loss/len_d, 100 * total_epoch_acc/len_d

In [ ]:
def test(model, loader):
    idx = 0
    accurcy = 0
    pres_rec = {
        'TP' : 0,
        'TN' : 0,
        'FP' : 0,
        'FN' : 0
    }
    label_dec = test_dataset.label_encoder.inverse_transform([0, 1, 2])
    model.cuda()
    model.eval()
    pbar = tqdm(loader)
    with torch.no_grad():
        for batch in pbar:
            emb1 = batch['sentence1']
            emb2 = batch['sentence2']
            target = batch['labels']
            if torch.cuda.is_available():
                emb1 = emb1.cuda()
                emb2 = emb2.cuda()
            prediction = model(emb1, emb2, batch_size=len(emb1))
            prediction = prediction.cpu()
            #inverse_transform ['contradiction', 'neutral', 'entailment'] self.label_encoder
            for i in range(len(prediction)):
                pr = label_dec[prediction[i].argmax().item()]
                tg = label_dec[target[i]]
                if pr == 'contradiction' and tg == 'contradiction':
                    pres_rec['FP'] += 1
                elif pr == 'contradiction' and tg != 'contradiction':
                    pres_rec['FN'] += 1
                elif pr != 'contradiction' and tg == 'contradiction':
                    pres_rec['TN'] += 1
                elif pr != 'contradiction' and tg != 'contradiction':
                    pres_rec['TP'] += 1

            accurcy += (torch.max(prediction, 1)[1].view(len(target)) == target).sum().item()
        
        return prediction, target, accurcy / len(emb1)

        

# Learning

In [ ]:
model = BiLSTM(batch_size, output_size, hidden_size, vocab_size, emb_dim)
criterion = nn.CrossEntropyLoss(reduction = 'sum')
opt = optim.Adam(model.parameters(), lr = 2e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=111, eta_min=0.000001)

In [ ]:
torch.cuda.empty_cache()
min_loss = 100
patience = 4
cur_patience = 0
history = {
    'train_loss' : [],
    'train_acc' : [],
    'val_loss' : [],
    'val_acc' : []
}
epoches = 15
for epoch in range(epoches):
    train_loss, train_acc = train(model, train_loader, epoch, criterion, opt)
    scheduler.step()
    val_loss, val_acc = eval(model, dev_loader, epoch, criterion)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
        cur_patience = 0
        torch.save(model.state_dict(), PATH_model)
        '''     
        if min_loss < 0.7:
            files.download(PATH_model)'''
    else:
        cur_patience += 1
        if cur_patience == patience:
            total_epoch = epoch
            break

    print(f'Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')


Train Loss: 0.661, Train Acc: 72.27%, Val. Loss: 0.544080, Val. Acc: 77.90%



Train Loss: 0.562, Train Acc: 77.64%, Val. Loss: 0.518667, Val. Acc: 79.76%



Train Loss: 0.536, Train Acc: 78.81%, Val. Loss: 0.502840, Val. Acc: 80.32%



Train Loss: 0.520, Train Acc: 79.66%, Val. Loss: 0.500221, Val. Acc: 80.64%



Train Loss: 0.507, Train Acc: 80.19%, Val. Loss: 0.499856, Val. Acc: 80.32%


...


Train Loss: 0.482, Train Acc: 81.26%, Val. Loss: 0.493993, Val. Acc: 80.84%



Train Loss: 0.478, Train Acc: 81.49%, Val. Loss: 0.489362, Val. Acc: 80.94%



Train Loss: 0.471, Train Acc: 81.77%, Val. Loss: 0.499535, Val. Acc: 80.85%



Train Loss: 0.467, Train Acc: 81.90%, Val. Loss: 0.496381, Val. Acc: 80.54%



Train Loss: 0.461, Train Acc: 82.23%, Val. Loss: 0.495933, Val. Acc: 80.94%


# Testing

In [ ]:
model = BiLSTM(batch_size, output_size, hidden_size, vocab_size, emb_dim)
model.load_state_dict(torch.load(PATH_model))
pred, target, acc = test(model, test_loader)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(target.tolist(), pred.argmax(dim=1).tolist()))

              precision    recall  f1-score   support

           0       0.85      0.78      0.81      3291
           1       0.82      0.83      0.82      3436
           2       0.74      0.79      0.76      3273

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000

